In [1]:
SAMPLING = 'weekly'
OUTFILE = f'../data/factor_pricing_data_{SAMPLING}.xlsx'

startdate = '19800101'
enddate = '20241231'

library = 'F-F_Research_Data_5_Factors_2x3'
library_momentum = 'F-F_Momentum_Factor'
library_ports = '49_Industry_Portfolios'

In [2]:
import pandas as pd
import pandas_datareader.data as web
import pandas_datareader as pdreader
pdreader.famafrench.get_available_datasets()

/Users/markhendricks/Projects/mlp-portfolio-mgmt-2025/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


['F-F_Research_Data_Factors',
 'F-F_Research_Data_Factors_weekly',
 'F-F_Research_Data_Factors_daily',
 'F-F_Research_Data_5_Factors_2x3',
 'F-F_Research_Data_5_Factors_2x3_daily',
 'Portfolios_Formed_on_ME',
 'Portfolios_Formed_on_ME_Wout_Div',
 'Portfolios_Formed_on_ME_Daily',
 'Portfolios_Formed_on_BE-ME',
 'Portfolios_Formed_on_BE-ME_Wout_Div',
 'Portfolios_Formed_on_BE-ME_Daily',
 'Portfolios_Formed_on_OP',
 'Portfolios_Formed_on_OP_Wout_Div',
 'Portfolios_Formed_on_OP_Daily',
 'Portfolios_Formed_on_INV',
 'Portfolios_Formed_on_INV_Wout_Div',
 'Portfolios_Formed_on_INV_Daily',
 '6_Portfolios_2x3',
 '6_Portfolios_2x3_Wout_Div',
 '6_Portfolios_2x3_weekly',
 '6_Portfolios_2x3_daily',
 '25_Portfolios_5x5',
 '25_Portfolios_5x5_Wout_Div',
 '25_Portfolios_5x5_Daily',
 '100_Portfolios_10x10',
 '100_Portfolios_10x10_Wout_Div',
 '100_Portfolios_10x10_Daily',
 '6_Portfolios_ME_OP_2x3',
 '6_Portfolios_ME_OP_2x3_Wout_Div',
 '6_Portfolios_ME_OP_2x3_daily',
 '25_Portfolios_ME_OP_5x5',
 '25_Portf

In [3]:
info = pd.DataFrame(columns=['Name', 'Unit', 'Construction', 'Description'])
info.loc['MKT'] = ['Market', 'Excess Return', 'Market-cap-weighted', 'US Equities']
info.loc['SMB'] = ['Size', 'Excess Return', 'Small Minus Big', 'Long small stocks and short big stocks']
info.loc['HML'] = ['Value', 'Excess Return', 'High Minus Low', 'Long value (high book-to-market) stocks and short growth (low book-to-market) stocks']
info.loc['RMW'] = ['Profitability','Excess Return', 'Robust Minus Weak', 'Long profitability (income statement) and short low profitability']
info.loc['CMA'] = ['Investment','Excess Return', 'Conservative Minus Agressive', 'Long stocks with low (conservative) investment (balance sheet) and short stocks with high (agressive) investment.']
info.loc['UMD'] = ['Momentum','Excess Return','Up Minus Down', 'Long stocks that have high recent returns and short stocks with low recent returns']
info.loc['RF'] = ['Risk-free rate', 'Total Return', 'Tbills', '']
info

,Name,Unit,Construction,Description
MKT,Market,Excess Return,Market-cap-weighted,US Equities
SMB,Size,Excess Return,Small Minus Big,Long small stocks and short big stocks
HML,Value,Excess Return,High Minus Low,Long value (high book-to-market) stocks and sh...
RMW,Profitability,Excess Return,Robust Minus Weak,Long profitability (income statement) and shor...
CMA,Investment,Excess Return,Conservative Minus Agressive,Long stocks with low (conservative) investment...
UMD,Momentum,Excess Return,Up Minus Down,Long stocks that have high recent returns and ...
RF,Risk-free rate,Total Return,Tbills,


In [4]:
if SAMPLING != 'monthly':
    library += '_daily'
    library_momentum += '_daily'
    library_ports += '_daily'

rawdata = web.DataReader(library, data_source='famafrench',start=startdate,end=enddate)
facs = rawdata[0] / 100
facs.rename(columns={'Mkt-RF':'MKT'},inplace=True)

rawdata = web.DataReader(library_momentum, data_source='famafrench',start=startdate,end=enddate)
umd = rawdata[0] / 100
umd.columns = ['UMD']

rawdata = web.DataReader(library_ports, data_source='famafrench',start=startdate, end=enddate)
rets = rawdata[0] / 100

/var/folders/zx/3v_qt0957xzg3nqtnkv007d00000gn/T/ipykernel_53600/2396518373.py:6: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  rawdata = web.DataReader(library, data_source='famafrench',start=startdate,end=enddate)
/var/folders/zx/3v_qt0957xzg3nqtnkv007d00000gn/T/ipykernel_53600/2396518373.py:10: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  rawdata = web.DataReader(library_momentum, data_source='famafrench',start=startdate,end=enddate)
/var/folders/zx/3v_qt0957xzg3nqtnkv007d00000gn/T/ipykernel_53600/2396518373.py:14: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' d

### Re-sample or Re-stamp

In [5]:
def cumulative_return(rets):
        cumret = (1 + rets).prod() - 1
        return cumret


if SAMPLING == 'monthly':
    # convert the yyyy-mm to timestamp object. default is to put first of month, but should be end of month
    facs = facs.to_timestamp().resample('M').last()
    umd = umd.to_timestamp().resample('M').last()
    rets = rets.to_timestamp().resample('M').last()


elif SAMPLING == 'weekly':
    facs = facs.resample('W-FRI').apply(cumulative_return)
    umd = umd.resample('W-FRI').apply(cumulative_return)
    rets = rets.resample('W-FRI').apply(cumulative_return)

In [6]:
rf = facs[['RF']]
facs = facs.join(umd).drop(columns=['RF'])
retsx = rets.subtract(rf['RF'],axis=0)

In [7]:
with pd.ExcelWriter(OUTFILE) as writer:  
    info.to_excel(writer, sheet_name= 'descriptions')
    facs.to_excel(writer, sheet_name='factors (excess returns)')
    retsx.to_excel(writer, sheet_name='portfolios (excess returns)')
    rf.to_excel(writer, sheet_name='risk-free rate')